In [1]:
import tensorflow as tf
import scipy 


### Model 1

model With arctan function and 80-20 split 

In [13]:
#80-20 split
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.8)]
train_ys = ys[:int(len(xs) * 0.8)]

val_xs = xs[-int(len(xs) * 0.2):]
val_ys = ys[-int(len(xs) * 0.2):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [14]:
# %load model.py
import tensorflow as tf
import scipy
class model():
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W, stride):
        return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

    x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
    y_ = tf.placeholder(tf.float32, shape=[None, 1])

    x_image = x

    #first convolutional layer
    W_conv1 = weight_variable([5, 5, 3, 24])
    b_conv1 = bias_variable([24])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

    #second convolutional layer
    W_conv2 = weight_variable([5, 5, 24, 36])
    b_conv2 = bias_variable([36])

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

    #third convolutional layer
    W_conv3 = weight_variable([5, 5, 36, 48])
    b_conv3 = bias_variable([48])

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

    #fourth convolutional layer
    W_conv4 = weight_variable([3, 3, 48, 64])
    b_conv4 = bias_variable([64])

    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

    #fifth convolutional layer
    W_conv5 = weight_variable([3, 3, 64, 64])
    b_conv5 = bias_variable([64])

    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

    #FCL 1
    W_fc1 = weight_variable([1152, 1164])
    b_fc1 = bias_variable([1164])

    h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #FCL 2
    W_fc2 = weight_variable([1164, 100])
    b_fc2 = bias_variable([100])

    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    #FCL 3
    W_fc3 = weight_variable([100, 50])
    b_fc3 = bias_variable([50])

    h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

    #FCL 3
    W_fc4 = weight_variable([50, 10])
    b_fc4 = bias_variable([10])

    h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

    h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

    #Output
    W_fc5 = weight_variable([10, 1])
    b_fc5 = bias_variable([1])

    y =  tf.multiply(tf.atan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output


In [4]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
#import driving_data
#import model

LOGDIR = './saveassign'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 40
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={model.x:xs,model.y_: ys, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
        #summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
        #summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


Instructions for updating:
Use `tf.global_variables_initializer` instead.


c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Epoch: 0, Step: 0, Loss: 6.34619
Epoch: 0, Step: 10, Loss: 6.29965
Epoch: 0, Step: 20, Loss: 6.1404
Epoch: 0, Step: 30, Loss: 6.02644
Epoch: 0, Step: 40, Loss: 5.9453
Epoch: 0, Step: 50, Loss: 5.95816
Epoch: 0, Step: 60, Loss: 6.08395
Epoch: 0, Step: 70, Loss: 6.94336
Epoch: 0, Step: 80, Loss: 7.60298
Epoch: 0, Step: 90, Loss: 6.00223
Epoch: 0, Step: 100, Loss: 5.68045
Epoch: 0, Step: 110, Loss: 5.60292
Epoch: 0, Step: 120, Loss: 5.54928
Epoch: 0, Step: 130, Loss: 5.53083
Epoch: 0, Step: 140, Loss: 5.50579
Epoch: 0, Step: 150, Loss: 5.53339
Epoch: 0, Step: 160, Loss: 5.44095
Epoch: 0, Step: 170, Loss: 5.3281
Epoch: 0, Step: 180, Loss: 5.37568
Epoch: 0, Step: 190, Loss: 5.57699
Epoch: 0, Step: 200, Loss: 5.22259
Epoch: 0, Step: 210, Loss: 5.26255
Epoch: 0, Step: 220, Loss: 5.09484
Epoch: 0, Step: 230, Loss: 5.05335
Epoch: 0, Step: 240, Loss: 5.01248
Epoch: 0, Step: 250, Loss: 4.97081
Epoch: 0, Step: 260, Loss: 4.96164
Epoch: 0, Step: 270, Loss: 9.45234
Epoch: 0, Step: 280, Loss: 5.65882

Model saved in file: ./saveassign\model.ckpt
Epoch: 5, Step: 500, Loss: 1.80999
Epoch: 5, Step: 510, Loss: 1.81068
Epoch: 5, Step: 520, Loss: 1.78645
Epoch: 5, Step: 530, Loss: 1.77297
Epoch: 5, Step: 540, Loss: 1.77515
Epoch: 5, Step: 550, Loss: 1.77377
Epoch: 5, Step: 560, Loss: 1.76179
Epoch: 5, Step: 570, Loss: 1.75537
Epoch: 5, Step: 580, Loss: 1.763
Epoch: 5, Step: 590, Loss: 1.79915
Epoch: 5, Step: 600, Loss: 4.4954
Epoch: 5, Step: 610, Loss: 4.63139
Epoch: 5, Step: 620, Loss: 1.72456
Epoch: 5, Step: 630, Loss: 1.71604
Epoch: 5, Step: 640, Loss: 1.73665
Epoch: 5, Step: 650, Loss: 1.73444
Epoch: 5, Step: 660, Loss: 1.72527
Epoch: 5, Step: 670, Loss: 1.73431
Epoch: 5, Step: 680, Loss: 1.77636
Epoch: 5, Step: 690, Loss: 1.68252
Epoch: 5, Step: 700, Loss: 1.68611
Epoch: 5, Step: 710, Loss: 1.72083
Epoch: 5, Step: 720, Loss: 1.79542
Epoch: 5, Step: 730, Loss: 1.79956
Epoch: 5, Step: 740, Loss: 1.6665
Epoch: 5, Step: 750, Loss: 1.66141
Epoch: 5, Step: 760, Loss: 1.7012
Epoch: 5, Step:

Epoch: 9, Step: 1320, Loss: 1.13322
Epoch: 9, Step: 1330, Loss: 0.970078
Epoch: 9, Step: 1340, Loss: 0.920225
Epoch: 9, Step: 1350, Loss: 0.871195
Model saved in file: ./saveassign\model.ckpt
Epoch: 10, Step: 1000, Loss: 0.944849
Epoch: 10, Step: 1010, Loss: 1.44337
Epoch: 10, Step: 1020, Loss: 2.15012
Epoch: 10, Step: 1030, Loss: 1.07362
Epoch: 10, Step: 1040, Loss: 0.954173
Epoch: 10, Step: 1050, Loss: 0.853042
Epoch: 10, Step: 1060, Loss: 0.856698
Epoch: 10, Step: 1070, Loss: 0.850452
Epoch: 10, Step: 1080, Loss: 0.851406
Epoch: 10, Step: 1090, Loss: 0.972562
Epoch: 10, Step: 1100, Loss: 1.1386
Epoch: 10, Step: 1110, Loss: 1.13238
Epoch: 10, Step: 1120, Loss: 0.892126
Epoch: 10, Step: 1130, Loss: 1.34016
Epoch: 10, Step: 1140, Loss: 1.26997
Epoch: 10, Step: 1150, Loss: 1.03963
Epoch: 10, Step: 1160, Loss: 0.832903
Epoch: 10, Step: 1170, Loss: 0.830338
Epoch: 10, Step: 1180, Loss: 0.849304
Epoch: 10, Step: 1190, Loss: 0.890117
Epoch: 10, Step: 1200, Loss: 2.08035
Epoch: 10, Step: 121

...

Epoch: 19, Step: 1950, Loss: 0.509261
Epoch: 19, Step: 1960, Loss: 0.562154
Epoch: 19, Step: 1970, Loss: 0.389672
Epoch: 19, Step: 1980, Loss: 0.38237
Epoch: 19, Step: 1990, Loss: 0.471718
Epoch: 19, Step: 2000, Loss: 0.586835
Epoch: 19, Step: 2010, Loss: 1.78732
Epoch: 19, Step: 2020, Loss: 0.746988
Epoch: 19, Step: 2030, Loss: 0.366995
Epoch: 19, Step: 2040, Loss: 0.430586
Epoch: 19, Step: 2050, Loss: 0.389983
Epoch: 19, Step: 2060, Loss: 0.581345
Epoch: 19, Step: 2070, Loss: 0.392017
Epoch: 19, Step: 2080, Loss: 0.537174
Epoch: 19, Step: 2090, Loss: 0.67072
Epoch: 19, Step: 2100, Loss: 0.373989
Epoch: 19, Step: 2110, Loss: 0.364819
Epoch: 19, Step: 2120, Loss: 0.364679
Epoch: 19, Step: 2130, Loss: 0.362364
Epoch: 19, Step: 2140, Loss: 0.362778
Epoch: 19, Step: 2150, Loss: 0.365656
Epoch: 19, Step: 2160, Loss: 0.365712
Epoch: 19, Step: 2170, Loss: 0.359686
Epoch: 19, Step: 2180, Loss: 0.369138
Epoch: 19, Step: 2190, Loss: 0.36557
Epoch: 19, Step: 2200, Loss: 0.361661
Epoch: 19, Step:

Epoch: 23, Step: 2630, Loss: 0.75738
Epoch: 23, Step: 2640, Loss: 0.406555
Epoch: 23, Step: 2650, Loss: 0.342971
Epoch: 23, Step: 2660, Loss: 0.298633
Epoch: 23, Step: 2670, Loss: 0.340169
Epoch: 23, Step: 2680, Loss: 0.321244
Epoch: 23, Step: 2690, Loss: 0.349385
Epoch: 23, Step: 2700, Loss: 0.325253
Epoch: 23, Step: 2710, Loss: 0.316461
Epoch: 23, Step: 2720, Loss: 0.328081
Epoch: 23, Step: 2730, Loss: 0.292609
Epoch: 23, Step: 2740, Loss: 0.292444
Epoch: 23, Step: 2750, Loss: 0.312426
Model saved in file: ./saveassign\model.ckpt
Epoch: 24, Step: 2400, Loss: 0.331249
Epoch: 24, Step: 2410, Loss: 0.352268
Epoch: 24, Step: 2420, Loss: 0.351085
Epoch: 24, Step: 2430, Loss: 1.29618
Epoch: 24, Step: 2440, Loss: 2.90467
Epoch: 24, Step: 2450, Loss: 2.67281
Epoch: 24, Step: 2460, Loss: 0.480691
Epoch: 24, Step: 2470, Loss: 0.357859
Epoch: 24, Step: 2480, Loss: 0.30534
Epoch: 24, Step: 2490, Loss: 0.302374
Epoch: 24, Step: 2500, Loss: 0.300963
Epoch: 24, Step: 2510, Loss: 0.29558
Epoch: 24, 

Epoch: 28, Step: 2950, Loss: 0.256548
Epoch: 28, Step: 2960, Loss: 0.254295
Epoch: 28, Step: 2970, Loss: 0.260807
Epoch: 28, Step: 2980, Loss: 0.25489
Epoch: 28, Step: 2990, Loss: 0.254883
Epoch: 28, Step: 3000, Loss: 0.250564
Epoch: 28, Step: 3010, Loss: 0.248822
Epoch: 28, Step: 3020, Loss: 0.266818
Epoch: 28, Step: 3030, Loss: 0.265846
Epoch: 28, Step: 3040, Loss: 0.26851
Epoch: 28, Step: 3050, Loss: 0.253443
Epoch: 28, Step: 3060, Loss: 0.24798
Epoch: 28, Step: 3070, Loss: 0.270399
Epoch: 28, Step: 3080, Loss: 0.30882
Epoch: 28, Step: 3090, Loss: 0.268073
Epoch: 28, Step: 3100, Loss: 0.249871
Epoch: 28, Step: 3110, Loss: 0.251075
Epoch: 28, Step: 3120, Loss: 0.251701
Epoch: 28, Step: 3130, Loss: 0.27351
Epoch: 28, Step: 3140, Loss: 0.267514
Epoch: 28, Step: 3150, Loss: 0.250975
Epoch: 28, Step: 3160, Loss: 0.253469
Epoch: 28, Step: 3170, Loss: 0.271611
Epoch: 28, Step: 3180, Loss: 0.283917
Epoch: 28, Step: 3190, Loss: 0.626103
Epoch: 28, Step: 3200, Loss: 0.817517
Epoch: 28, Step: 

Epoch: 32, Step: 3630, Loss: 0.64722
Epoch: 32, Step: 3640, Loss: 0.366252
Epoch: 32, Step: 3650, Loss: 0.251069
Model saved in file: ./saveassign\model.ckpt
Epoch: 33, Step: 3300, Loss: 0.235405
Epoch: 33, Step: 3310, Loss: 0.372228
Epoch: 33, Step: 3320, Loss: 0.251193
Epoch: 33, Step: 3330, Loss: 0.280235
Epoch: 33, Step: 3340, Loss: 0.678465
Epoch: 33, Step: 3350, Loss: 0.523263
Epoch: 33, Step: 3360, Loss: 0.530037
Epoch: 33, Step: 3370, Loss: 0.491626
Epoch: 33, Step: 3380, Loss: 0.329468
Epoch: 33, Step: 3390, Loss: 0.284793
Epoch: 33, Step: 3400, Loss: 0.271625
Epoch: 33, Step: 3410, Loss: 0.238168
Epoch: 33, Step: 3420, Loss: 0.251392
Epoch: 33, Step: 3430, Loss: 0.2312
Epoch: 33, Step: 3440, Loss: 0.220084
Epoch: 33, Step: 3450, Loss: 0.219925
Epoch: 33, Step: 3460, Loss: 0.218973
Epoch: 33, Step: 3470, Loss: 0.227664
Epoch: 33, Step: 3480, Loss: 0.224635
Epoch: 33, Step: 3490, Loss: 0.225082
Epoch: 33, Step: 3500, Loss: 0.225629
Epoch: 33, Step: 3510, Loss: 0.230246
Epoch: 3

Epoch: 37, Step: 3950, Loss: 0.305378
Epoch: 37, Step: 3960, Loss: 0.216419
Epoch: 37, Step: 3970, Loss: 0.230904
Epoch: 37, Step: 3980, Loss: 0.217319
Epoch: 37, Step: 3990, Loss: 0.260944
Epoch: 37, Step: 4000, Loss: 0.287552
Epoch: 37, Step: 4010, Loss: 0.269449
Epoch: 37, Step: 4020, Loss: 0.357691
Epoch: 37, Step: 4030, Loss: 0.596779
Epoch: 37, Step: 4040, Loss: 0.271232
Epoch: 37, Step: 4050, Loss: 0.202452
Epoch: 37, Step: 4060, Loss: 0.20058
Epoch: 37, Step: 4070, Loss: 0.204907
Epoch: 37, Step: 4080, Loss: 0.225296
Epoch: 37, Step: 4090, Loss: 0.250059
Epoch: 37, Step: 4100, Loss: 0.200995
Epoch: 37, Step: 4110, Loss: 0.730293
Epoch: 37, Step: 4120, Loss: 1.57181
Epoch: 37, Step: 4130, Loss: 0.612119
Epoch: 37, Step: 4140, Loss: 0.411272
Epoch: 37, Step: 4150, Loss: 0.238629
Model saved in file: ./saveassign\model.ckpt
Epoch: 38, Step: 3800, Loss: 0.223611
Epoch: 38, Step: 3810, Loss: 0.217923
Epoch: 38, Step: 3820, Loss: 0.212048
Epoch: 38, Step: 3830, Loss: 0.203452
Epoch: 

### Model 2

Model with Linear function,dropout of 0.5 and 70-30 data split

In [1]:
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [2]:
# %load model.py
import tensorflow as tf
import scipy
# model with dropout 0.5 
class model_2():
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W, stride):
        return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

    x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
    y_ = tf.placeholder(tf.float32, shape=[None, 1])

    x_image = x

    #first convolutional layer
    W_conv1 = weight_variable([5, 5, 3, 24])
    b_conv1 = bias_variable([24])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

    #second convolutional layer
    W_conv2 = weight_variable([5, 5, 24, 36])
    b_conv2 = bias_variable([36])

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

    #third convolutional layer
    W_conv3 = weight_variable([5, 5, 36, 48])
    b_conv3 = bias_variable([48])

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

    #fourth convolutional layer
    W_conv4 = weight_variable([3, 3, 48, 64])
    b_conv4 = bias_variable([64])

    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

    #fifth convolutional layer
    W_conv5 = weight_variable([3, 3, 64, 64])
    b_conv5 = bias_variable([64])

    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

    #FCL 1
    W_fc1 = weight_variable([1152, 1164])
    b_fc1 = bias_variable([1164])

    h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #FCL 2
    W_fc2 = weight_variable([1164, 100])
    b_fc2 = bias_variable([100])

    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    #FCL 3
    W_fc3 = weight_variable([100, 50])
    b_fc3 = bias_variable([50])

    h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

    #FCL 3
    W_fc4 = weight_variable([50, 10])
    b_fc4 = bias_variable([10])

    h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

    h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

    #Output
    W_fc5 = weight_variable([10, 1])
    b_fc5 = bias_variable([1])

    y =  tf.matmul(h_fc4_drop, W_fc5) + b_fc5 #scale the atan output


In [2]:
# %load model.py
import tensorflow as tf
import scipy
# model with dropout 0.5 
class model_3():
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W, stride):
        return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

    x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
    y_ = tf.placeholder(tf.float32, shape=[None, 1])

    x_image = x

    #first convolutional layer
    W_conv1 = weight_variable([5, 5, 3, 24])
    b_conv1 = bias_variable([24])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

    #second convolutional layer
    W_conv2 = weight_variable([5, 5, 24, 36])
    b_conv2 = bias_variable([36])

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

    #third convolutional layer
    W_conv3 = weight_variable([5, 5, 36, 48])
    b_conv3 = bias_variable([48])

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

    #fourth convolutional layer
    W_conv4 = weight_variable([3, 3, 48, 64])
    b_conv4 = bias_variable([64])

    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

    #fifth convolutional layer
    W_conv5 = weight_variable([3, 3, 64, 64])
    b_conv5 = bias_variable([64])

    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

    #FCL 1
    W_fc1 = weight_variable([1152, 1164])
    b_fc1 = bias_variable([1164])

    h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #FCL 2
    W_fc2 = weight_variable([1164, 100])
    b_fc2 = bias_variable([100])

    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    #FCL 3
    W_fc3 = weight_variable([100, 50])
    b_fc3 = bias_variable([50])

    h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

    #FCL 3
    W_fc4 = weight_variable([50, 10])
    b_fc4 = bias_variable([10])

    h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

    h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

    #Output
    W_fc5 = weight_variable([10, 1])
    b_fc5 = bias_variable([1])

    y =  tf.matmul(h_fc4_drop, W_fc5) + b_fc5 #scale the atan output


In [37]:
sess.run(tf.global_variables_initializer())
sess.close()

In [4]:
#model with learning rate of 1e-3
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
#import driving_data
#import model

LOGDIR = './saveassign4'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model_3.y_, model_3.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={model_3.x: xs, model_3.y_: ys, model_3.keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={model_3.x:xs,model_3.y_: ys, model_3.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model_3.x:xs, model_3.y_: ys, model_3.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:48: Deprecation

Epoch: 0, Step: 0, Loss: 6.32723
Epoch: 0, Step: 10, Loss: 5.67481
Epoch: 0, Step: 20, Loss: 5.37282
Epoch: 0, Step: 30, Loss: 5.13092
Epoch: 0, Step: 40, Loss: 4.916
Epoch: 0, Step: 50, Loss: 4.72568
Epoch: 0, Step: 60, Loss: 4.55786
Epoch: 0, Step: 70, Loss: 4.40938
Epoch: 0, Step: 80, Loss: 4.27745
Epoch: 0, Step: 90, Loss: 4.16907
Epoch: 0, Step: 100, Loss: 4.05589
Epoch: 0, Step: 110, Loss: 3.95733
Epoch: 0, Step: 120, Loss: 3.86557
Epoch: 0, Step: 130, Loss: 3.80209
Epoch: 0, Step: 140, Loss: 3.75853
Epoch: 0, Step: 150, Loss: 3.95049
Epoch: 0, Step: 160, Loss: 3.62075
Epoch: 0, Step: 170, Loss: 3.54454
Epoch: 0, Step: 180, Loss: 3.54105
Epoch: 0, Step: 190, Loss: 3.65402
Epoch: 0, Step: 200, Loss: 3.51535
Epoch: 0, Step: 210, Loss: 3.32264
Epoch: 0, Step: 220, Loss: 3.52557
Epoch: 0, Step: 230, Loss: 3.53384
Epoch: 0, Step: 240, Loss: 3.17686
Epoch: 0, Step: 250, Loss: 3.16767
Epoch: 0, Step: 260, Loss: 3.08007
Epoch: 0, Step: 270, Loss: 3.07969
Epoch: 0, Step: 280, Loss: 3.0228

Model saved in file: ./saveassign4\model.ckpt
Epoch: 5, Step: 500, Loss: 0.887533
Epoch: 5, Step: 510, Loss: 0.882043
Epoch: 5, Step: 520, Loss: 0.876133
Epoch: 5, Step: 530, Loss: 0.873328
Epoch: 5, Step: 540, Loss: 0.870994
Epoch: 5, Step: 550, Loss: 0.867778
Epoch: 5, Step: 560, Loss: 0.869142
Epoch: 5, Step: 570, Loss: 0.892606
Epoch: 5, Step: 580, Loss: 0.875736
Epoch: 5, Step: 590, Loss: 0.849235
Epoch: 5, Step: 600, Loss: 0.843589
Epoch: 5, Step: 610, Loss: 0.841271
Epoch: 5, Step: 620, Loss: 0.836794
Epoch: 5, Step: 630, Loss: 0.831655
Epoch: 5, Step: 640, Loss: 0.829843
Epoch: 5, Step: 650, Loss: 0.824148
Epoch: 5, Step: 660, Loss: 0.820915
Epoch: 5, Step: 670, Loss: 0.818644
Epoch: 5, Step: 680, Loss: 0.814311
Epoch: 5, Step: 690, Loss: 0.810398
Epoch: 5, Step: 700, Loss: 0.80624
Epoch: 5, Step: 710, Loss: 0.802691
Epoch: 5, Step: 720, Loss: 0.798915
Epoch: 5, Step: 730, Loss: 0.796784
Epoch: 5, Step: 740, Loss: 0.845734
Epoch: 5, Step: 750, Loss: 0.982721
Epoch: 5, Step: 760

Epoch: 9, Step: 1270, Loss: 0.885506
Epoch: 9, Step: 1280, Loss: 1.39905
Epoch: 9, Step: 1290, Loss: 0.408941
Epoch: 9, Step: 1300, Loss: 0.311044
Epoch: 9, Step: 1310, Loss: 0.500892
Epoch: 9, Step: 1320, Loss: 1.62873
Epoch: 9, Step: 1330, Loss: 0.944536
Epoch: 9, Step: 1340, Loss: 0.422054
Epoch: 9, Step: 1350, Loss: 0.32767
Model saved in file: ./saveassign4\model.ckpt
Epoch: 10, Step: 1000, Loss: 0.38208
Epoch: 10, Step: 1010, Loss: 0.679393
Epoch: 10, Step: 1020, Loss: 0.520318
Epoch: 10, Step: 1030, Loss: 1.48249
Epoch: 10, Step: 1040, Loss: 1.03627
Epoch: 10, Step: 1050, Loss: 0.335906
Epoch: 10, Step: 1060, Loss: 0.300409
Epoch: 10, Step: 1070, Loss: 0.308714
Epoch: 10, Step: 1080, Loss: 0.314865
Epoch: 10, Step: 1090, Loss: 0.345447
Epoch: 10, Step: 1100, Loss: 0.857695
Epoch: 10, Step: 1110, Loss: 0.795678
Epoch: 10, Step: 1120, Loss: 0.424862
Epoch: 10, Step: 1130, Loss: 0.318193
Epoch: 10, Step: 1140, Loss: 0.298562
Epoch: 10, Step: 1150, Loss: 0.320243
Epoch: 10, Step: 11

Epoch: 14, Step: 1580, Loss: 0.102689
Epoch: 14, Step: 1590, Loss: 0.155578
Epoch: 14, Step: 1600, Loss: 2.53957
Epoch: 14, Step: 1610, Loss: 3.11493
Epoch: 14, Step: 1620, Loss: 0.0972921
Epoch: 14, Step: 1630, Loss: 0.0940549
Epoch: 14, Step: 1640, Loss: 0.150238
Epoch: 14, Step: 1650, Loss: 0.157556
Epoch: 14, Step: 1660, Loss: 0.1473
Epoch: 14, Step: 1670, Loss: 0.135041
Epoch: 14, Step: 1680, Loss: 0.203412
Epoch: 14, Step: 1690, Loss: 0.0940068
Epoch: 14, Step: 1700, Loss: 0.113773
Epoch: 14, Step: 1710, Loss: 0.112566
Epoch: 14, Step: 1720, Loss: 0.1041
Epoch: 14, Step: 1730, Loss: 0.0974167
Epoch: 14, Step: 1740, Loss: 0.176139
Epoch: 14, Step: 1750, Loss: 0.182436
Epoch: 14, Step: 1760, Loss: 0.173843
Epoch: 14, Step: 1770, Loss: 0.220607
Epoch: 14, Step: 1780, Loss: 0.0990174
Epoch: 14, Step: 1790, Loss: 0.111669
Epoch: 14, Step: 1800, Loss: 0.100837
Epoch: 14, Step: 1810, Loss: 0.0977118
Epoch: 14, Step: 1820, Loss: 0.0971293
Epoch: 14, Step: 1830, Loss: 0.0968663
Epoch: 14,

Epoch: 18, Step: 2230, Loss: 0.465161
Epoch: 18, Step: 2240, Loss: 1.35264
Epoch: 18, Step: 2250, Loss: 0.404105
Model saved in file: ./saveassign4\model.ckpt
Epoch: 19, Step: 1900, Loss: 0.0270344
Epoch: 19, Step: 1910, Loss: 0.0389524
Epoch: 19, Step: 1920, Loss: 0.0539323
Epoch: 19, Step: 1930, Loss: 0.0623467
Epoch: 19, Step: 1940, Loss: 0.281719
Epoch: 19, Step: 1950, Loss: 0.617651
Epoch: 19, Step: 1960, Loss: 0.319654
Epoch: 19, Step: 1970, Loss: 0.11974
Epoch: 19, Step: 1980, Loss: 0.0697016
Epoch: 19, Step: 1990, Loss: 0.0333759
Epoch: 19, Step: 2000, Loss: 0.124496
Epoch: 19, Step: 2010, Loss: 0.783286
Epoch: 19, Step: 2020, Loss: 1.48264
Epoch: 19, Step: 2030, Loss: 0.299607
Epoch: 19, Step: 2040, Loss: 0.145089
Epoch: 19, Step: 2050, Loss: 0.0241053
Epoch: 19, Step: 2060, Loss: 0.0280481
Epoch: 19, Step: 2070, Loss: 0.0239604
Epoch: 19, Step: 2080, Loss: 0.0353331
Epoch: 19, Step: 2090, Loss: 0.0597655
Epoch: 19, Step: 2100, Loss: 0.0666108
Epoch: 19, Step: 2110, Loss: 0.02

Epoch: 23, Step: 2500, Loss: 0.0822602
Epoch: 23, Step: 2510, Loss: 0.0172534
Epoch: 23, Step: 2520, Loss: 0.0933017
Epoch: 23, Step: 2530, Loss: 0.083518
Epoch: 23, Step: 2540, Loss: 0.00684136
Epoch: 23, Step: 2550, Loss: 0.0528236
Epoch: 23, Step: 2560, Loss: 0.0170591
Epoch: 23, Step: 2570, Loss: 0.0198768
Epoch: 23, Step: 2580, Loss: 0.00967939
Epoch: 23, Step: 2590, Loss: 0.156318
Epoch: 23, Step: 2600, Loss: 0.0460735
Epoch: 23, Step: 2610, Loss: 0.134102
Epoch: 23, Step: 2620, Loss: 0.0796272
Epoch: 23, Step: 2630, Loss: 0.0382459
Epoch: 23, Step: 2640, Loss: 0.0245942
Epoch: 23, Step: 2650, Loss: 0.0258997
Epoch: 23, Step: 2660, Loss: 0.0240148
Epoch: 23, Step: 2670, Loss: 0.0246404
Epoch: 23, Step: 2680, Loss: 0.0235037
Epoch: 23, Step: 2690, Loss: 0.021417
Epoch: 23, Step: 2700, Loss: 0.0199525
Epoch: 23, Step: 2710, Loss: 0.0354254
Epoch: 23, Step: 2720, Loss: 0.0308652
Epoch: 23, Step: 2730, Loss: 0.0183899
Epoch: 23, Step: 2740, Loss: 0.0175205
Epoch: 23, Step: 2750, Loss

Epoch: 27, Step: 3130, Loss: 0.0256577
Epoch: 27, Step: 3140, Loss: 0.0344104
Epoch: 27, Step: 3150, Loss: 0.461893
Model saved in file: ./saveassign4\model.ckpt
Epoch: 28, Step: 2800, Loss: 0.616253
Epoch: 28, Step: 2810, Loss: 0.208351
Epoch: 28, Step: 2820, Loss: 0.0451068
Epoch: 28, Step: 2830, Loss: 0.0344056
Epoch: 28, Step: 2840, Loss: 0.0203789
Epoch: 28, Step: 2850, Loss: 0.160444
Epoch: 28, Step: 2860, Loss: 1.21367
Epoch: 28, Step: 2870, Loss: 1.20403
Epoch: 28, Step: 2880, Loss: 0.0523455
Epoch: 28, Step: 2890, Loss: 0.0943784
Epoch: 28, Step: 2900, Loss: 0.00251545
Epoch: 28, Step: 2910, Loss: 0.00733563
Epoch: 28, Step: 2920, Loss: 0.00904558
Epoch: 28, Step: 2930, Loss: 0.0233353
Epoch: 28, Step: 2940, Loss: 0.0582764
Epoch: 28, Step: 2950, Loss: 0.018825
Epoch: 28, Step: 2960, Loss: 0.0122184
Epoch: 28, Step: 2970, Loss: 0.120538
Epoch: 28, Step: 2980, Loss: 0.416092
Epoch: 28, Step: 2990, Loss: 0.209288
Epoch: 28, Step: 3000, Loss: 0.0347551
Epoch: 28, Step: 3010, Loss

In [4]:
#model with learning rate of 1e-4
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
#import driving_data
#import model

LOGDIR = './saveassign3'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model_2.y_, model_2.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 35
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={model_2.x: xs, model_2.y_: ys, model_2.keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={model_2.x:xs,model_2.y_: ys, model_2.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model_2.x:xs, model_2.y_: ys, model_2.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


Instructions for updating:
Use `tf.global_variables_initializer` instead.


c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Epoch: 0, Step: 0, Loss: 13.9995
Epoch: 0, Step: 10, Loss: 12.1772
Epoch: 0, Step: 20, Loss: 12.0587
Epoch: 0, Step: 30, Loss: 11.914
Epoch: 0, Step: 40, Loss: 12.018
Epoch: 0, Step: 50, Loss: 11.5795
Epoch: 0, Step: 60, Loss: 11.5813
Epoch: 0, Step: 70, Loss: 11.6402
Epoch: 0, Step: 80, Loss: 11.3706
Epoch: 0, Step: 90, Loss: 11.026
Epoch: 0, Step: 100, Loss: 10.8737
Epoch: 0, Step: 110, Loss: 10.7131
Epoch: 0, Step: 120, Loss: 10.5998
Epoch: 0, Step: 130, Loss: 10.7651
Epoch: 0, Step: 140, Loss: 11.1177
Epoch: 0, Step: 150, Loss: 10.4858
Epoch: 0, Step: 160, Loss: 10.9171
Epoch: 0, Step: 170, Loss: 10.1996
Epoch: 0, Step: 180, Loss: 10.4612
Epoch: 0, Step: 190, Loss: 10.0729
Epoch: 0, Step: 200, Loss: 9.60928
Epoch: 0, Step: 210, Loss: 9.47807
Epoch: 0, Step: 220, Loss: 9.40136
Epoch: 0, Step: 230, Loss: 9.37512
Epoch: 0, Step: 240, Loss: 10.2145
Epoch: 0, Step: 250, Loss: 9.2264
Epoch: 0, Step: 260, Loss: 10.9168
Epoch: 0, Step: 270, Loss: 9.37029
Epoch: 0, Step: 280, Loss: 9.89346


Model saved in file: ./saveassign3\model.ckpt
Epoch: 5, Step: 500, Loss: 1.83182
Epoch: 5, Step: 510, Loss: 1.82496
Epoch: 5, Step: 520, Loss: 1.81505
Epoch: 5, Step: 530, Loss: 1.80657
Epoch: 5, Step: 540, Loss: 1.79738
Epoch: 5, Step: 550, Loss: 1.78871
Epoch: 5, Step: 560, Loss: 1.77823
Epoch: 5, Step: 570, Loss: 1.77064
Epoch: 5, Step: 580, Loss: 1.77422
Epoch: 5, Step: 590, Loss: 1.80966
Epoch: 5, Step: 600, Loss: 4.56873
Epoch: 5, Step: 610, Loss: 4.55276
Epoch: 5, Step: 620, Loss: 1.72784
Epoch: 5, Step: 630, Loss: 1.71617
Epoch: 5, Step: 640, Loss: 1.75271
Epoch: 5, Step: 650, Loss: 1.74723
Epoch: 5, Step: 660, Loss: 1.72853
Epoch: 5, Step: 670, Loss: 1.74287
Epoch: 5, Step: 680, Loss: 1.80299
Epoch: 5, Step: 690, Loss: 1.6674
Epoch: 5, Step: 700, Loss: 1.68702
Epoch: 5, Step: 710, Loss: 1.67277
Epoch: 5, Step: 720, Loss: 1.65556
Epoch: 5, Step: 730, Loss: 1.64118
Epoch: 5, Step: 740, Loss: 1.70954
Epoch: 5, Step: 750, Loss: 1.68957
Epoch: 5, Step: 760, Loss: 1.70662
Epoch: 5, 

Epoch: 9, Step: 1300, Loss: 0.891644
Epoch: 9, Step: 1310, Loss: 1.11422
Epoch: 9, Step: 1320, Loss: 0.893358
Epoch: 9, Step: 1330, Loss: 0.734285
Epoch: 9, Step: 1340, Loss: 0.680331
Epoch: 9, Step: 1350, Loss: 0.656695
Model saved in file: ./saveassign3\model.ckpt
Epoch: 10, Step: 1000, Loss: 0.75339
Epoch: 10, Step: 1010, Loss: 1.40951
Epoch: 10, Step: 1020, Loss: 2.06272
Epoch: 10, Step: 1030, Loss: 0.858828
Epoch: 10, Step: 1040, Loss: 0.718295
Epoch: 10, Step: 1050, Loss: 0.632811
Epoch: 10, Step: 1060, Loss: 0.63822
Epoch: 10, Step: 1070, Loss: 0.63069
Epoch: 10, Step: 1080, Loss: 0.635361
Epoch: 10, Step: 1090, Loss: 0.656288
Epoch: 10, Step: 1100, Loss: 0.665204
Epoch: 10, Step: 1110, Loss: 0.621102
Epoch: 10, Step: 1120, Loss: 0.72092
Epoch: 10, Step: 1130, Loss: 0.998931
Epoch: 10, Step: 1140, Loss: 0.700696
Epoch: 10, Step: 1150, Loss: 0.779717
Epoch: 10, Step: 1160, Loss: 0.608517
Epoch: 10, Step: 1170, Loss: 0.607758
Epoch: 10, Step: 1180, Loss: 0.604846
Epoch: 10, Step: 

Epoch: 14, Step: 1610, Loss: 0.389692
Epoch: 14, Step: 1620, Loss: 0.332066
Epoch: 14, Step: 1630, Loss: 0.291986
Epoch: 14, Step: 1640, Loss: 0.27704
Epoch: 14, Step: 1650, Loss: 0.275112
Epoch: 14, Step: 1660, Loss: 0.274959
Epoch: 14, Step: 1670, Loss: 0.275956
Epoch: 14, Step: 1680, Loss: 0.277699
Epoch: 14, Step: 1690, Loss: 0.272107
Epoch: 14, Step: 1700, Loss: 0.26213
Epoch: 14, Step: 1710, Loss: 0.260023
Epoch: 14, Step: 1720, Loss: 0.25405
Epoch: 14, Step: 1730, Loss: 0.267769
Epoch: 14, Step: 1740, Loss: 0.299835
Epoch: 14, Step: 1750, Loss: 0.252046
Epoch: 14, Step: 1760, Loss: 0.268816
Epoch: 14, Step: 1770, Loss: 0.640672
Epoch: 14, Step: 1780, Loss: 0.254477
Epoch: 14, Step: 1790, Loss: 0.298814
Epoch: 14, Step: 1800, Loss: 0.299026
Epoch: 14, Step: 1810, Loss: 0.479005
Epoch: 14, Step: 1820, Loss: 0.268282
Epoch: 14, Step: 1830, Loss: 0.334056
Epoch: 14, Step: 1840, Loss: 0.432822
Epoch: 14, Step: 1850, Loss: 0.443862
Model saved in file: ./saveassign3\model.ckpt
Epoch: 

Epoch: 19, Step: 1920, Loss: 0.158051
Epoch: 19, Step: 1930, Loss: 0.161465
Epoch: 19, Step: 1940, Loss: 0.234135
Epoch: 19, Step: 1950, Loss: 0.165037
Epoch: 19, Step: 1960, Loss: 0.162466
Epoch: 19, Step: 1970, Loss: 0.24799
Epoch: 19, Step: 1980, Loss: 0.564747
Epoch: 19, Step: 1990, Loss: 0.44992
Epoch: 19, Step: 2000, Loss: 0.174572
Epoch: 19, Step: 2010, Loss: 0.144618
Epoch: 19, Step: 2020, Loss: 0.149122
Epoch: 19, Step: 2030, Loss: 0.153813
Epoch: 19, Step: 2040, Loss: 0.157246
Epoch: 19, Step: 2050, Loss: 0.447905
Epoch: 19, Step: 2060, Loss: 5.14071
Epoch: 19, Step: 2070, Loss: 0.191558
Epoch: 19, Step: 2080, Loss: 0.148091
Epoch: 19, Step: 2090, Loss: 0.143209
Epoch: 19, Step: 2100, Loss: 0.147344
Epoch: 19, Step: 2110, Loss: 0.147043
Epoch: 19, Step: 2120, Loss: 0.143415
Epoch: 19, Step: 2130, Loss: 0.150632
Epoch: 19, Step: 2140, Loss: 0.144387
Epoch: 19, Step: 2150, Loss: 0.148442
Epoch: 19, Step: 2160, Loss: 0.147143
Epoch: 19, Step: 2170, Loss: 0.159009
Epoch: 19, Step

Epoch: 23, Step: 2600, Loss: 0.16463
Epoch: 23, Step: 2610, Loss: 0.280317
Epoch: 23, Step: 2620, Loss: 0.2859
Epoch: 23, Step: 2630, Loss: 0.188239
Epoch: 23, Step: 2640, Loss: 0.160168
Epoch: 23, Step: 2650, Loss: 0.184086
Epoch: 23, Step: 2660, Loss: 0.19332
Epoch: 23, Step: 2670, Loss: 0.129776
Epoch: 23, Step: 2680, Loss: 0.160391
Epoch: 23, Step: 2690, Loss: 0.209495
Epoch: 23, Step: 2700, Loss: 0.303952
Epoch: 23, Step: 2710, Loss: 0.280301
Epoch: 23, Step: 2720, Loss: 0.120374
Epoch: 23, Step: 2730, Loss: 0.16818
Epoch: 23, Step: 2740, Loss: 0.137113
Epoch: 23, Step: 2750, Loss: 0.699657
Model saved in file: ./saveassign3\model.ckpt
Epoch: 24, Step: 2400, Loss: 0.470258
Epoch: 24, Step: 2410, Loss: 0.705569
Epoch: 24, Step: 2420, Loss: 0.890965
Epoch: 24, Step: 2430, Loss: 0.198127
Epoch: 24, Step: 2440, Loss: 0.610243
Epoch: 24, Step: 2450, Loss: 0.237318
Epoch: 24, Step: 2460, Loss: 0.123293
Epoch: 24, Step: 2470, Loss: 0.14382
Epoch: 24, Step: 2480, Loss: 0.267865
Epoch: 24,

Epoch: 28, Step: 2910, Loss: 2.85923
Epoch: 28, Step: 2920, Loss: 0.120278
Epoch: 28, Step: 2930, Loss: 0.105898
Epoch: 28, Step: 2940, Loss: 0.106488
Epoch: 28, Step: 2950, Loss: 0.112844
Epoch: 28, Step: 2960, Loss: 0.105286
Epoch: 28, Step: 2970, Loss: 0.107947
Epoch: 28, Step: 2980, Loss: 0.107136
Epoch: 28, Step: 2990, Loss: 0.103476
Epoch: 28, Step: 3000, Loss: 0.103833
Epoch: 28, Step: 3010, Loss: 0.103416
Epoch: 28, Step: 3020, Loss: 0.114252
Epoch: 28, Step: 3030, Loss: 0.14108
Epoch: 28, Step: 3040, Loss: 0.126941
Epoch: 28, Step: 3050, Loss: 0.105417
Epoch: 28, Step: 3060, Loss: 0.103662
Epoch: 28, Step: 3070, Loss: 0.11729
Epoch: 28, Step: 3080, Loss: 0.112896
Epoch: 28, Step: 3090, Loss: 0.117705
Epoch: 28, Step: 3100, Loss: 0.105475
Epoch: 28, Step: 3110, Loss: 0.115983
Epoch: 28, Step: 3120, Loss: 0.126401
Epoch: 28, Step: 3130, Loss: 0.327683
Epoch: 28, Step: 3140, Loss: 3.92927
Epoch: 28, Step: 3150, Loss: 0.645843
Epoch: 28, Step: 3160, Loss: 0.307754
Epoch: 28, Step:

Epoch: 32, Step: 3590, Loss: 0.127194
Epoch: 32, Step: 3600, Loss: 0.428041
Epoch: 32, Step: 3610, Loss: 0.245501
Epoch: 32, Step: 3620, Loss: 0.453529
Epoch: 32, Step: 3630, Loss: 1.49393
Epoch: 32, Step: 3640, Loss: 0.918596
Epoch: 32, Step: 3650, Loss: 0.387467
Model saved in file: ./saveassign3\model.ckpt
Epoch: 33, Step: 3300, Loss: 0.172357
Epoch: 33, Step: 3310, Loss: 0.116013
Epoch: 33, Step: 3320, Loss: 0.107265
Epoch: 33, Step: 3330, Loss: 0.14448
Epoch: 33, Step: 3340, Loss: 0.640497
Epoch: 33, Step: 3350, Loss: 0.437393
Epoch: 33, Step: 3360, Loss: 1.36234
Epoch: 33, Step: 3370, Loss: 1.04592
Epoch: 33, Step: 3380, Loss: 0.973118
Epoch: 33, Step: 3390, Loss: 0.249872
Epoch: 33, Step: 3400, Loss: 0.105135
Epoch: 33, Step: 3410, Loss: 0.189316
Epoch: 33, Step: 3420, Loss: 1.18661
Epoch: 33, Step: 3430, Loss: 1.38041
Epoch: 33, Step: 3440, Loss: 0.137827
Epoch: 33, Step: 3450, Loss: 0.143361
Epoch: 33, Step: 3460, Loss: 0.139996
Epoch: 33, Step: 3470, Loss: 0.54979
Epoch: 33, 

In [17]:
len(train_xs)

36324

In [11]:
#pip3 install opencv-python

import tensorflow as tf
import scipy.misc
#import model
import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "saveassign3/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model_2.y.eval(feed_dict={model_2.x: [image], model_2.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()



INFO:tensorflow:Restoring parameters from saveassign3/model.ckpt
Starting frameofvideo:31785
Steering angle: -4.043464304684733 (pred)	-28.34 (actual)
Steering angle: -4.96236551221969 (pred)	-28.840000000000003 (actual)
Steering angle: -3.954957859646061 (pred)	-29.75 (actual)
Steering angle: -1.439905902989415 (pred)	-31.06 (actual)
Steering angle: 0.25931069316047284 (pred)	-32.27 (actual)
Steering angle: 0.5473397704599106 (pred)	-33.48 (actual)
Steering angle: -0.8502590870367992 (pred)	-34.39 (actual)

c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.



Steering angle: -0.9919940500510263 (pred)	-35.599999999999994 (actual)
Steering angle: -0.5946943257492552 (pred)	-36.5 (actual)
Steering angle: -0.5850727236428229 (pred)	-37.61 (actual)
Steering angle: -0.8339063335033193 (pred)	-38.62 (actual)
Steering angle: -0.20073541772526662 (pred)	-39.63 (actual)
Steering angle: -0.04204109500189786 (pred)	-39.93000000000001 (actual)
Steering angle: -0.4775915862030328 (pred)	-40.03000000000001 (actual)
Steering angle: -0.7887301816723856 (pred)	-40.03000000000001 (actual)
Steering angle: -0.9547515898278855 (pred)	-40.03000000000001 (actual)
Steering angle: -0.7912065521333383 (pred)	-40.03000000000001 (actual)
Steering angle: -0.16672363697954187 (pred)	-40.03000000000001 (actual)
Steering angle: 0.4224832721018428 (pred)	-40.03000000000001 (actual)
Steering angle: 0.668708390029629 (pred)	-40.03000000000001 (actual)
Steering angle: 1.4266741190198078 (pred)	-40.03000000000001 (actual)
Steering angle: 1.8659859100195042 (pred)	-40.24 (actu

Steering angle: -0.8874994128258243 (pred)	-0.3 (actual)
Steering angle: -0.7979547790333109 (pred)	0.2 (actual)
Steering angle: -0.3028160099656922 (pred)	0.3 (actual)
Steering angle: 0.3760401201798717 (pred)	0.4 (actual)
Steering angle: -2.3571507896506447 (pred)	0.5 (actual)
Steering angle: -2.7123317255487094 (pred)	1.01 (actual)
Steering angle: -1.204505567679041 (pred)	1.6100000000000003 (actual)
Steering angle: -0.25087583642233025 (pred)	2.12 (actual)
Steering angle: 0.48397311732194037 (pred)	2.52 (actual)
Steering angle: 0.5434405862174795 (pred)	3.13 (actual)
Steering angle: 0.5468960216072712 (pred)	4.13 (actual)
Steering angle: 0.7484314252390757 (pred)	4.64 (actual)
Steering angle: 0.8207319658114225 (pred)	4.94 (actual)
Steering angle: 0.9948601681814929 (pred)	4.94 (actual)
Steering angle: 1.1393141868156198 (pred)	4.94 (actual)
Steering angle: 1.49098611302759 (pred)	4.94 (actual)
Steering angle: 2.535585212848997 (pred)	4.84 (actual)
Steering angle: 1.767615177715453

Steering angle: 5.197980571602097 (pred)	21.78 (actual)
Steering angle: 5.608804924234139 (pred)	21.68 (actual)
Steering angle: 5.220620941153396 (pred)	21.68 (actual)
Steering angle: 4.796496075773224 (pred)	21.68 (actual)
Steering angle: 5.563376909177561 (pred)	0.0 (actual)
Steering angle: 3.6086753795760536 (pred)	22.69 (actual)
Steering angle: 3.223856758765423 (pred)	23.189999999999998 (actual)
Steering angle: 3.4714946586343443 (pred)	23.39 (actual)
Steering angle: 4.564889056773145 (pred)	23.6 (actual)
Steering angle: 10.700247778275553 (pred)	24.1 (actual)
Steering angle: 12.018615160739646 (pred)	24.5 (actual)
Steering angle: 13.315273640700498 (pred)	24.810000000000002 (actual)
Steering angle: 14.22132811618031 (pred)	24.91 (actual)
Steering angle: 14.70939977389108 (pred)	0.0 (actual)
Steering angle: 13.566324927607134 (pred)	24.91 (actual)
Steering angle: 15.882126178909346 (pred)	24.91 (actual)
Steering angle: 14.423006526897861 (pred)	25.11 (actual)
Steering angle: 13.67

...

Steering angle: 6.4404150335158645 (pred)	9.58 (actual)
Steering angle: 7.88653085912129 (pred)	9.78 (actual)
Steering angle: 7.6161603821472745 (pred)	9.979999999999999 (actual)
Steering angle: 5.051258716720114 (pred)	10.18 (actual)
Steering angle: 3.1047572561040298 (pred)	10.18 (actual)
Steering angle: 3.573469809829458 (pred)	10.18 (actual)
Steering angle: 4.259698702297228 (pred)	10.389999999999999 (actual)
Steering angle: 5.999825997141376 (pred)	11.089999999999998 (actual)
Steering angle: 6.657653468936352 (pred)	11.5 (actual)
Steering angle: 7.790628172985142 (pred)	11.8 (actual)
Steering angle: 8.003050471267123 (pred)	12.0 (actual)
Steering angle: 7.4594177859792365 (pred)	12.300000000000002 (actual)
Steering angle: 6.424315850755321 (pred)	12.61 (actual)
Steering angle: 5.2113673154884985 (pred)	0.0 (actual)
Steering angle: 4.934285323881299 (pred)	12.910000000000002 (actual)
Steering angle: 4.526882896023432 (pred)	12.910000000000002 (actual)
Steering angle: 4.437038160794

Steering angle: 4.573092540853153 (pred)	-1.11 (actual)
Steering angle: 4.766835552415619 (pred)	-1.11 (actual)
Steering angle: 4.630856304437597 (pred)	-1.11 (actual)
Steering angle: 4.664833507350649 (pred)	-1.11 (actual)
Steering angle: 4.656615936005478 (pred)	-1.21 (actual)
Steering angle: 4.479448939784986 (pred)	-1.21 (actual)
Steering angle: 4.638600885182746 (pred)	0.0 (actual)
Steering angle: 4.576938791129517 (pred)	-2.32 (actual)
Steering angle: 4.511772810032071 (pred)	-2.32 (actual)
Steering angle: 4.722817971422649 (pred)	-1.92 (actual)
Steering angle: 4.769771253098256 (pred)	-0.61 (actual)
Steering angle: 4.946548928536056 (pred)	0.5 (actual)
Steering angle: 4.900121571426541 (pred)	1.92 (actual)
Steering angle: 5.010908093537794 (pred)	2.22 (actual)
Steering angle: 4.875291699359429 (pred)	2.22 (actual)
Steering angle: 5.069290843015771 (pred)	2.22 (actual)
Steering angle: 4.905842708630073 (pred)	2.22 (actual)
Steering angle: 4.856899280585054 (pred)	2.22 (actual)
St

Steering angle: 7.081456461651888 (pred)	17.45 (actual)
Steering angle: 7.293388693860921 (pred)	17.75 (actual)
Steering angle: 7.868950805971324 (pred)	17.95 (actual)
Steering angle: 8.386888319601622 (pred)	18.149999999999995 (actual)
Steering angle: 8.485574307824551 (pred)	18.45 (actual)
Steering angle: 8.878567170967806 (pred)	18.760000000000005 (actual)
Steering angle: 9.21072268540011 (pred)	19.26 (actual)
Steering angle: 9.209811708919561 (pred)	19.66 (actual)
Steering angle: 9.740599369699591 (pred)	19.87 (actual)
Steering angle: 10.06340435515815 (pred)	20.17 (actual)
Steering angle: 10.30210922136634 (pred)	20.57 (actual)
Steering angle: 10.008086653070217 (pred)	20.869999999999997 (actual)
Steering angle: 9.900430918919767 (pred)	21.079999999999995 (actual)
Steering angle: 9.086902454805182 (pred)	21.38 (actual)
Steering angle: 8.781372725304879 (pred)	21.78 (actual)
Steering angle: 8.367130289880068 (pred)	22.08 (actual)
Steering angle: 8.410247566563065 (pred)	22.69 (actu

Steering angle: 2.393325605929148 (pred)	2.22 (actual)
Steering angle: 2.564570401252415 (pred)	2.22 (actual)
Steering angle: 2.5983693789168116 (pred)	2.22 (actual)
Steering angle: 2.7076818608278113 (pred)	0.0 (actual)


IndexError: list index out of range

## Conclusion:
    

Objective:
To build a model that predicts the angle at which the steering wheel of the car is rotated.

1. Size: 25 minutes = 25#60#30 = 45,000 images ~ 2.3 GB
<br>
2. Firstly We check if the train and test distrubutions are same.
<br>
2. We split the data in 70(train) 30(test),convert the images into shape of [66,200,3] and load the data in the batches.
<br>
3. We build a CNN model and train the model in batches and evaluate the loss on test data. 
<br>
4. We try to minimize the squares loss and save the model checkpoint.
<br>
5. We then perform hyperparameter tuning : we try different dropouts ,train/test split and learning rate of adam optimizer
<br>
6. We visualize the output using opencv


with learning rate =1e-3 the output was wrong ,hence used 1e-4 itself